In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6610,2021-01-03T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1137,184,1321,34270,...,2494,NaN,NaN,96547,1238041,823898.0,NaN,11.0,NaN,NaN
6611,2021-01-03T17:00:00,ITA,9,Toscana,43.769231,11.255889,834,147,981,8967,...,3730,NaN,NaN,121770,1904826,1069499.0,NaN,11.0,Positivi diagnosticati solo con test antigenic...,NaN
6612,2021-01-03T17:00:00,ITA,10,Umbria,43.106758,12.388247,268,48,316,3588,...,631,NaN,NaN,29440,509101,254304.0,NaN,3.0,NaN,NaN
6613,2021-01-03T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,66,2,68,346,...,386,NaN,NaN,7332,63369,38354.0,NaN,0.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6610,2021-01-03T17:00:00,19,Sicilia,1137,184,1321,34270,35591,641,1047,58462,2494,NaN,NaN,96547,1238041,823898.0,11.0,NaN,NaN
6611,2021-01-03T17:00:00,9,Toscana,834,147,981,8967,9948,-73,355,108092,3730,NaN,NaN,121770,1904826,1069499.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN
6612,2021-01-03T17:00:00,10,Umbria,268,48,316,3588,3904,111,202,24905,631,NaN,NaN,29440,509101,254304.0,3.0,NaN,NaN
6613,2021-01-03T17:00:00,2,Valle d'Aosta,66,2,68,346,414,1,35,6532,386,NaN,NaN,7332,63369,38354.0,0.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-01-03', '2021-01-02')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-03T17:00:00,1,Piemonte,2786,188,2974,18489,21463,-2734,741,170618,7979,NaN,NaN,200060,1694353,990328.0,13.0,NaN,NaN
1,2021-01-03T17:00:00,2,Valle d'Aosta,66,2,68,346,414,1,35,6532,386,NaN,NaN,7332,63369,38354.0,0.0,NaN,NaN
2,2021-01-03T17:00:00,3,Lombardia,3267,489,3756,53199,56955,945,1709,402798,25317,NaN,NaN,485070,4910001,2626398.0,12.0,NaN,NaN
3,2021-01-03T17:00:00,5,Veneto,2688,365,3053,92106,95159,1038,3419,163342,6763,NaN,NaN,265264,3348514,1228798.0,19.0,NaN,NaN
4,2021-01-03T17:00:00,6,Friuli Venezia Giulia,658,61,719,10930,11649,-198,200,37945,1707,NaN,NaN,51301,945021,344372.0,6.0,NaN,NaN
5,2021-01-03T17:00:00,7,Liguria,696,71,767,5020,5787,85,221,52606,2912,NaN,NaN,61305,717946,325350.0,6.0,I 164106 test antigenici rapidi sono riferiti ...,NaN
6,2021-01-03T17:00:00,8,Emilia-Romagna,2657,234,2891,53737,56628,-1878,1818,113473,7887,NaN,NaN,177988,2603180,1239887.0,15.0,NaN,NaN
7,2021-01-03T17:00:00,9,Toscana,834,147,981,8967,9948,-73,355,108092,3730,NaN,NaN,121770,1904826,1069499.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2021-01-03T17:00:00,10,Umbria,268,48,316,3588,3904,111,202,24905,631,NaN,NaN,29440,509101,254304.0,3.0,NaN,NaN
9,2021-01-03T17:00:00,11,Marche,479,67,546,11520,12066,279,372,29301,1607,NaN,NaN,42974,542199,321308.0,8.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-02T17:00:00,1,Piemonte,2777,182,2959,21238,24197,-2332,433,167151,7971,NaN,NaN,199319,1690587,988567.0,0.0,NaN,NaN
1,2021-01-02T17:00:00,2,Valle d'Aosta,76,2,78,335,413,5,7,6501,383,NaN,NaN,7297,63032,37893.0,2.0,NaN,NaN
2,2021-01-02T17:00:00,3,Lombardia,3293,491,3784,52226,56010,698,1402,402070,25281,NaN,NaN,483361,4896792,2621183.0,20.0,NaN,NaN
3,2021-01-02T17:00:00,5,Veneto,2668,372,3040,91081,94121,1539,3165,161049,6675,NaN,NaN,261845,3334350,1224449.0,15.0,NaN,NaN
4,2021-01-02T17:00:00,6,Friuli Venezia Giulia,652,63,715,11132,11847,94,359,37567,1687,NaN,NaN,51101,942810,343713.0,4.0,NaN,NaN
5,2021-01-02T17:00:00,7,Liguria,674,66,740,4962,5702,105,203,52475,2907,NaN,NaN,61084,715806,324658.0,5.0,I 162885 test antigenici rapidi sono riferiti ...,NaN
6,2021-01-02T17:00:00,8,Emilia-Romagna,2649,229,2878,55628,58506,1719,2035,109818,7846,NaN,NaN,176170,2588793,1234681.0,17.0,NaN,NaN
7,2021-01-02T17:00:00,9,Toscana,825,139,964,9057,10021,185,498,107674,3720,NaN,NaN,121415,1898439,1066811.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2021-01-02T17:00:00,10,Umbria,270,47,317,3476,3793,-11,50,24817,628,NaN,NaN,29238,506539,253466.0,1.0,NaN,NaN
9,2021-01-02T17:00:00,11,Marche,479,65,544,11243,11787,48,285,29221,1594,NaN,NaN,42602,539624,319908.0,4.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-03T17:00:00,1,Piemonte,2786,188,2974,18489,21463,-2734,741,170618,7979,NaN,NaN,200060,1694353,990328.0,13.0,NaN,NaN
1,2021-01-03T17:00:00,2,Valle d'Aosta,66,2,68,346,414,1,35,6532,386,NaN,NaN,7332,63369,38354.0,0.0,NaN,NaN
2,2021-01-03T17:00:00,3,Lombardia,3267,489,3756,53199,56955,945,1709,402798,25317,NaN,NaN,485070,4910001,2626398.0,12.0,NaN,NaN
3,2021-01-03T17:00:00,4,Trentino-Alto Adige,568,68,636,12195,12831,11,226,37682,1712,0.0,0.0,52225,821187,316043.0,5.0,NaN,NaN
4,2021-01-03T17:00:00,5,Veneto,2688,365,3053,92106,95159,1038,3419,163342,6763,NaN,NaN,265264,3348514,1228798.0,19.0,NaN,NaN
5,2021-01-03T17:00:00,6,Friuli Venezia Giulia,658,61,719,10930,11649,-198,200,37945,1707,NaN,NaN,51301,945021,344372.0,6.0,NaN,NaN
6,2021-01-03T17:00:00,7,Liguria,696,71,767,5020,5787,85,221,52606,2912,NaN,NaN,61305,717946,325350.0,6.0,I 164106 test antigenici rapidi sono riferiti ...,NaN
7,2021-01-03T17:00:00,8,Emilia-Romagna,2657,234,2891,53737,56628,-1878,1818,113473,7887,NaN,NaN,177988,2603180,1239887.0,15.0,NaN,NaN
8,2021-01-03T17:00:00,9,Toscana,834,147,981,8967,9948,-73,355,108092,3730,NaN,NaN,121770,1904826,1069499.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2021-01-03T17:00:00,10,Umbria,268,48,316,3588,3904,111,202,24905,631,NaN,NaN,29440,509101,254304.0,3.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2021-01-02T17:00:00,1,Piemonte,2777,182,2959,21238,24197,-2332,...,167151,7971,NaN,NaN,199319,1690587,988567.0,0.0,NaN,NaN
1,1,2021-01-02T17:00:00,2,Valle d'Aosta,76,2,78,335,413,5,...,6501,383,NaN,NaN,7297,63032,37893.0,2.0,NaN,NaN
2,2,2021-01-02T17:00:00,3,Lombardia,3293,491,3784,52226,56010,698,...,402070,25281,NaN,NaN,483361,4896792,2621183.0,20.0,NaN,NaN
3,19,2021-01-02T17:00:00,4,Trentino-Alto Adige,545,65,610,12210,12820,128,...,37475,1704,0.0,0.0,51999,818332,315473.0,3.0,0,NaN
4,3,2021-01-02T17:00:00,5,Veneto,2668,372,3040,91081,94121,1539,...,161049,6675,NaN,NaN,261845,3334350,1224449.0,15.0,NaN,NaN
5,4,2021-01-02T17:00:00,6,Friuli Venezia Giulia,652,63,715,11132,11847,94,...,37567,1687,NaN,NaN,51101,942810,343713.0,4.0,NaN,NaN
6,5,2021-01-02T17:00:00,7,Liguria,674,66,740,4962,5702,105,...,52475,2907,NaN,NaN,61084,715806,324658.0,5.0,I 162885 test antigenici rapidi sono riferiti ...,NaN
7,6,2021-01-02T17:00:00,8,Emilia-Romagna,2649,229,2878,55628,58506,1719,...,109818,7846,NaN,NaN,176170,2588793,1234681.0,17.0,NaN,NaN
8,7,2021-01-02T17:00:00,9,Toscana,825,139,964,9057,10021,185,...,107674,3720,NaN,NaN,121415,1898439,1066811.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2021-01-02T17:00:00,10,Umbria,270,47,317,3476,3793,-11,...,24817,628,NaN,NaN,29238,506539,253466.0,1.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2021-01-03T17:00:00,1,Piemonte,2786,188,2974,18489,21463,-2734,741,...,7979,NaN,NaN,200060,1694353,990328.0,13.0,NaN,NaN,4341375
1,2021-01-03T17:00:00,2,Valle d'Aosta,66,2,68,346,414,1,35,...,386,NaN,NaN,7332,63369,38354.0,0.0,NaN,NaN,125501
2,2021-01-03T17:00:00,3,Lombardia,3267,489,3756,53199,56955,945,1709,...,25317,NaN,NaN,485070,4910001,2626398.0,12.0,NaN,NaN,10103969
3,2021-01-03T17:00:00,4,Trentino-Alto Adige,568,68,636,12195,12831,11,226,...,1712,0.0,0.0,52225,821187,316043.0,5.0,NaN,NaN,1074819
4,2021-01-03T17:00:00,5,Veneto,2688,365,3053,92106,95159,1038,3419,...,6763,NaN,NaN,265264,3348514,1228798.0,19.0,NaN,NaN,4907704
5,2021-01-03T17:00:00,6,Friuli Venezia Giulia,658,61,719,10930,11649,-198,200,...,1707,NaN,NaN,51301,945021,344372.0,6.0,NaN,NaN,1211357
6,2021-01-03T17:00:00,7,Liguria,696,71,767,5020,5787,85,221,...,2912,NaN,NaN,61305,717946,325350.0,6.0,I 164106 test antigenici rapidi sono riferiti ...,NaN,1543127
7,2021-01-03T17:00:00,8,Emilia-Romagna,2657,234,2891,53737,56628,-1878,1818,...,7887,NaN,NaN,177988,2603180,1239887.0,15.0,NaN,NaN,4467118
8,2021-01-03T17:00:00,9,Toscana,834,147,981,8967,9948,-73,355,...,3730,NaN,NaN,121770,1904826,1069499.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2021-01-03T17:00:00,10,Umbria,268,48,316,3588,3904,111,202,...,631,NaN,NaN,29440,509101,254304.0,3.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2021-01-03T17:00:00,1,Piemonte,2786,188,2974,18489,21463,-2734,741,...,NaN,NaN,200060,1694353,990328.0,13.0,NaN,NaN,4341375,575
1,2021-01-03T17:00:00,2,Valle d'Aosta,66,2,68,346,414,1,35,...,NaN,NaN,7332,63369,38354.0,0.0,NaN,NaN,125501,20
2,2021-01-03T17:00:00,3,Lombardia,3267,489,3756,53199,56955,945,1709,...,NaN,NaN,485070,4910001,2626398.0,12.0,NaN,NaN,10103969,1036
3,2021-01-03T17:00:00,4,Trentino-Alto Adige,568,68,636,12195,12831,11,226,...,0.0,0.0,52225,821187,316043.0,5.0,NaN,NaN,1074819,106
4,2021-01-03T17:00:00,5,Veneto,2688,365,3053,92106,95159,1038,3419,...,NaN,NaN,265264,3348514,1228798.0,19.0,NaN,NaN,4907704,1016
5,2021-01-03T17:00:00,6,Friuli Venezia Giulia,658,61,719,10930,11649,-198,200,...,NaN,NaN,51301,945021,344372.0,6.0,NaN,NaN,1211357,180
6,2021-01-03T17:00:00,7,Liguria,696,71,767,5020,5787,85,221,...,NaN,NaN,61305,717946,325350.0,6.0,I 164106 test antigenici rapidi sono riferiti ...,NaN,1543127,209
7,2021-01-03T17:00:00,8,Emilia-Romagna,2657,234,2891,53737,56628,-1878,1818,...,NaN,NaN,177988,2603180,1239887.0,15.0,NaN,NaN,4467118,634
8,2021-01-03T17:00:00,9,Toscana,834,147,981,8967,9948,-73,355,...,NaN,NaN,121770,1904826,1069499.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2021-01-03T17:00:00,10,Umbria,268,48,316,3588,3904,111,202,...,NaN,NaN,29440,509101,254304.0,3.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-01-03T17:00:00,5,Veneto,2688,365,3053,92106,95159,1038,3419,...,35.93,1025,254,14164,4349.0,88,2293,24.14,0.06967,5.41
1,2021-01-03T17:00:00,8,Emilia-Romagna,2657,234,2891,53737,56628,-1878,1818,...,36.91,-1891,-217,14387,5206.0,41,3655,12.64,0.04070,3.98
2,2021-01-03T17:00:00,3,Lombardia,3267,489,3756,53199,56955,945,1709,...,47.20,973,307,13209,5215.0,36,728,12.94,0.01691,4.80
3,2021-01-03T17:00:00,12,Lazio,2824,309,3133,74020,77153,705,1681,...,36.48,679,406,11892,6015.0,24,952,14.14,0.02866,2.86
4,2021-01-03T17:00:00,19,Sicilia,1137,184,1321,34270,35591,641,1047,...,31.29,596,313,6319,3798.0,26,380,16.57,0.02107,1.94
5,2021-01-03T17:00:00,16,Puglia,1476,136,1612,52466,54078,266,950,...,36.86,296,606,7591,2437.0,25,659,12.51,0.02370,2.34
6,2021-01-03T17:00:00,1,Piemonte,2786,188,2974,18489,21463,-2734,741,...,32.70,-2749,308,3766,1761.0,8,3467,19.68,0.01707,4.61
7,2021-01-03T17:00:00,15,Campania,1362,98,1460,76365,77825,-190,619,...,19.41,-196,227,6411,4219.0,18,791,9.66,0.01070,3.33
8,2021-01-03T17:00:00,11,Marche,479,67,546,11520,12066,279,372,...,46.85,277,87,2575,1400.0,13,80,14.45,0.02450,2.83
9,2021-01-03T17:00:00,9,Toscana,834,147,981,8967,9948,-73,355,...,28.11,-90,-143,6387,2688.0,10,418,5.56,0.00954,3.27


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,68,38,106,64.15
1,Lombardia,489,547,1036,47.20
2,Marche,67,76,143,46.85
3,Umbria,48,69,117,41.03
4,Emilia-Romagna,234,400,634,36.91
5,Puglia,136,233,369,36.86
6,Lazio,309,538,847,36.48
7,Veneto,365,651,1016,35.93
8,Liguria,71,138,209,33.97
9,Friuli Venezia Giulia,61,119,180,33.89


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2021-01-01T17:00:00,22,P.A. Trento,362,39,401,1498,1899,37,193,19182,952,NaN,NaN,22033,453381,151838.0,0.0,NaN,"Dei 193 casi odierni, 46 sono molecolari con p..."
626,2021-01-02T17:00:00,21,P.A. Bolzano,176,24,200,10638,10838,45,86,18266,746,NaN,NaN,29850,364372,163511.0,0.0,NaN,NaN
627,2021-01-02T17:00:00,22,P.A. Trento,369,41,410,1572,1982,83,116,19209,958,NaN,NaN,22149,453960,151962.0,3.0,NaN,"Dei 116 casi odierni, 57 sono molecolari con p..."
628,2021-01-03T17:00:00,21,P.A. Bolzano,186,24,210,10640,10850,12,96,18345,751,NaN,NaN,29946,365076,163696.0,1.0,Nota del Ministero della Salute: i casi positi...,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,2020-12-30T17:00:00,4,Trentino-Alto Adige,548,67,615,11700,12315,-123,417,...,9.0,5003.0,1074.0,-154.0,10.89,4.99,8.33,38.83,-1.47,-0.99
311,2020-12-31T17:00:00,4,Trentino-Alto Adige,539,68,607,11924,12531,216,580,...,17.0,4598.0,957.0,224.0,11.20,4.84,12.61,60.61,1.49,1.75
312,2021-01-01T17:00:00,4,Trentino-Alto Adige,521,66,587,12105,12692,161,463,...,13.0,4978.0,960.0,181.0,11.24,4.62,9.30,48.23,-2.94,1.28
313,2021-01-02T17:00:00,4,Trentino-Alto Adige,545,65,610,12210,12820,128,202,...,10.0,1302.0,359.0,105.0,10.66,4.76,15.51,56.27,-1.52,1.01


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
